# Libraries

In [1]:
import re
import random
from pathlib import Path
from scipy.sparse import dok_matrix
from constants import *
from utils import get_text

In [2]:
data_path = Path("./data/Shakespeare")

# Character-wise tokenization

## Data

In [3]:
def tokenize_text(text: str) -> list:
    """
    Given text, returns it as list of tokens.
    """
    return list(text)


def textify_tokens(tokens: list) -> str:
    """
    Given list of tokens, returns them as text str.
    """
    return ''.join(tokens)

In [4]:
text = get_text([data_path])

text_as_tokens = tokenize_text(text)
tokens = list(set(text_as_tokens))
tokens.sort()
vocab_size = len(tokens)
token_to_id = {word: idx for idx, word in enumerate(tokens)}

print(f"Corpus size: {len(text_as_tokens)} \t Unique tokens: {len(tokens)}")

Corpus size: 1215376 	 Unique tokens: 65


## k-matrix
Matrix that maps last k-words to the immediate next word.  
Suppose we were taking words as tokens.  
Example: for text, "Let it be" and `k=4`, the matrix would look like:
```text
.           "L"   "e"  "t"  "i"  "b"  " "
["Let "] :   0     0    0    1    0    0
["et i"] :   0     0    1    0    0    0
["t it"] :   0     0    0    0    0    1
[" it "] :   0     0    0    0    1    0
["it b"] :   0     1    0    0    0    0
["t be"] :   0     0    0    0    0    0
```
Of course there are other mappings possible, but above ones are those present in the example sentence.  
Since our matrix is going to be sparse we use `scipy.dok_matrix`.  


In [5]:
def get_k_matrix(k: int):
    """
    Creates a matrix that maps conditional probability of next word in the sentence,
    given past k words in the sentence.
    :param k: past-k words to be considered
    :return:
    """
    if k < 1:
        raise ValueError("k should be greater than 0.")
    seq_of_k = [''.join(text_as_tokens[i:i+k]) for i in range(len(text_as_tokens)-k)]
    set_seq_of_k = list(set(seq_of_k))
    set_seq_of_k.sort()
    seq_of_k_to_id = {s: idx for idx, s in enumerate(set_seq_of_k)}
    k_matrix = dok_matrix((len(set_seq_of_k), vocab_size))
    for i, last_k in enumerate(seq_of_k[:-k]):
        last_k_words_row = seq_of_k_to_id[last_k]
        next_word_col = token_to_id[text_as_tokens[i + k]]
        k_matrix[last_k_words_row, next_word_col] += 1
    return k_matrix, seq_of_k_to_id

## Text generation

In [6]:
def sample_next_token_after_seq(k_matrix, seq_of_k_to_id: dict, seq: str, alpha: float = 0) -> str:
    """
    Given sequence of last k tokens, samples next token
    :param seq:
    :param alpha: Creativity hyperparameter, large the value => distribution approches uniormity => choice of random token
    :return:
    """
    if seq not in seq_of_k_to_id:
        # markov chains has not seen any following token to this sequence
        return random.choice(tokens)
    next_token_vector = k_matrix[seq_of_k_to_id[seq]] + alpha
    likelihoods = next_token_vector/next_token_vector.sum()
    return random.choices(tokens, likelihoods.toarray()[0])[0]

def stochastic_chain(k_matrix, seq_of_k_to_id: dict, seed: str = None, chain_length: int = 15):
    """
    Returns sentence with added tokens equal to 'chain_length'.
    The last k-token phrase of the seed sentence need to be present in some part of the training dataset.

    :param seed: If None, randomly generates seed sentence.
    :param chain_length: Added tokens to be generated.
    :return: seed sentence + generated sentence.
    """
    k = len(list(seq_of_k_to_id.keys())[0])
    if seed is None:
        seed = random.choice(list(seq_of_k_to_id.keys()))
    current_tokens = tokenize_text(seed)
    if len(current_tokens) < k:
        raise ValueError(f"Seed length must be at-least as long as {k}")
    sentence = seed
    current_tokens = current_tokens[-k:]
    if seq_of_k_to_id.get(textify_tokens(current_tokens), None) is None:
        raise ValueError("The last k-word phrase of the seed sentence need to be present in some part of the training dataset.")
    for _ in range(chain_length):
        next_token = sample_next_token_after_seq(k_matrix, seq_of_k_to_id, seq=textify_tokens(current_tokens))
        sentence += next_token
        current_tokens = current_tokens[1:] + [next_token]
    return sentence

In [7]:
k_list = list(range(1, 15))
# k_list = [7]
k_calculated = {}
for k in k_list:
    k_matrix, seq_of_k_to_id = get_k_matrix(k=k)
    print(f"{len(seq_of_k_to_id)}  {k}-token-sequences have been mapped to {vocab_size} tokens.")
    k_calculated[k] = (k_matrix, seq_of_k_to_id)

65  1-token-sequences have been mapped to 65 tokens.
1481  2-token-sequences have been mapped to 65 tokens.
12249  3-token-sequences have been mapped to 65 tokens.
53249  4-token-sequences have been mapped to 65 tokens.
148302  5-token-sequences have been mapped to 65 tokens.
300238  6-token-sequences have been mapped to 65 tokens.
478117  7-token-sequences have been mapped to 65 tokens.
656104  8-token-sequences have been mapped to 65 tokens.
812085  9-token-sequences have been mapped to 65 tokens.
933034  10-token-sequences have been mapped to 65 tokens.
1020602  11-token-sequences have been mapped to 65 tokens.
1081144  12-token-sequences have been mapped to 65 tokens.
1121676  13-token-sequences have been mapped to 65 tokens.
1148692  14-token-sequences have been mapped to 65 tokens.


In [8]:
for k, calculated in k_calculated.items():
    k_matrix, seq_of_k_to_id = calculated
    generated = stochastic_chain(
        k_matrix=k_matrix,
        seq_of_k_to_id=seq_of_k_to_id,
        seed=None,
        chain_length=100
    )
    print(f"\n------------------------- GENERATED SENTENCE -- k = {k}")
    print(generated)
    print("------------------------- END")


------------------------- GENERATED SENTENCE -- k = 1
oul mestothald schersheang y mens f
Any this
My INIsheff gemashelly waderearleisweinde towat s lleavi
------------------------- END

------------------------- GENERATED SENTENCE -- k = 2
EVAUREL:
Supoke the not night hiceiring offech beare eveign prale and, ing and mortessio.

Dide and th
------------------------- END

------------------------- GENERATED SENTENCE -- k = 3
kitch
The grow; but morship.

LUCIO:
I with alls,
To Baption ship clost that your quick moth to thou ho
------------------------- END

------------------------- GENERATED SENTENCE -- k = 4
? Prithmetimes not be sorrow the for than hope it for life,--for one's sometime bettery March times?

AR
------------------------- END

------------------------- GENERATED SENTENCE -- k = 5
ere England what's o'clock?

BRUTUS:
Do not so,
That walk in her nature,
The more; I'll desire that atten
------------------------- END

------------------------- GENERATED SENTENCE -- k = 6


# Word-wise tokenization
Taking each word as a token.

## Data

In [3]:
def tokenize_text(text: str) -> list:
    """
    Given text, returns it as list of tokens.
    """
    tokens = process_text(text=text)
    tokens = tokens.split(' ')
    tokens = list(filter(lambda x: x != '', tokens))
    return tokens


def textify_tokens(tokens: list) -> str:
    """
    Given list of tokens, returns them as text str.
    """
    return ' '.join(tokens)


puncs = {
    '\n': '\n', '\t': '\t', '.': '. ', '-': '-', ',': ', ', '!': '! ', '?': '?' ,
    '“': '“', '”': '” ',  '(': '(', '—': '—', ')': ') ', '/': '/', '\'': '\'', ';': '; ',
    ':': ': ', '‘': '‘', '$': '$'
}


def process_text(text: str) -> str:
    """
    Inserts space before and after punctuations.
    :param text:
    :return: cleaned text
    """
    cleaned_text = text
    for punc in puncs:
        cleaned_text = cleaned_text.replace(punc, f' {punc} ')
    cleaned_text = re.sub("[ ]+", " ", cleaned_text)
    return cleaned_text


def postprocess_text(text: str) -> str:
    """
    Tries to unddo opposite of the processings done on input.
    """
    cleaned_text = text
    for punc, pre_punc in puncs.items():
        cleaned_text = cleaned_text.replace(f' {punc}', pre_punc)
    cleaned_text = re.sub("[ ]+", " ", cleaned_text)
    return cleaned_text

In [4]:
text = get_text([data_path])
text_as_tokens = tokenize_text(text)
tokens = list(set(text_as_tokens))
tokens.sort()
vocab_size = len(tokens)
token_to_id = {word: idx for idx, word in enumerate(tokens)}

print(f"Corpus size: {len(text_as_tokens)} \t Unique tokens: {len(tokens)}")

Corpus size: 329579 	 Unique tokens: 14343


## k-matrix
Matrix that maps last k-words to the immediate next word.  
Example: for text, "To be or not to be" and `k=4`, the matrix would look like:
```text
.                            "To"   "be"  "not"  "or"  "to"
["To", "be", "or", "not"] :   0      0     0      0     1
["be", "or", "not", "to"] :   0      1     0      0     0
["or", "not", "to", "be"] :   0      0     0      0     0
```
Of course there are other mappings possible, but above ones are those present in the example sentence.  
Since our matrix is going to be sparse we use `scipy.dok_matrix`.  


In [5]:
def get_k_matrix(k: int):
    """
    Creates a matrix that maps conditional probability of next word in the sentence,
    given past k words in the sentence.
    :param k: past-k words to be considered
    :return:
    """
    if k < 1:
        raise ValueError("k should be greater than 0.")
    seq_of_k = [' '.join(text_as_tokens[i:i+k]) for i in range(len(text_as_tokens)-k)]
    set_seq_of_k = list(set(seq_of_k))
    set_seq_of_k.sort()
    seq_of_k_to_id = {s: idx for idx, s in enumerate(set_seq_of_k)}
    k_matrix = dok_matrix((len(set_seq_of_k), vocab_size))
    for i, last_k in enumerate(seq_of_k[:-k]):
        last_k_words_row = seq_of_k_to_id[last_k]
        next_word_col = token_to_id[text_as_tokens[i + k]]
        k_matrix[last_k_words_row, next_word_col] += 1
    return k_matrix, seq_of_k_to_id

## Text generation

In [6]:
def sample_next_token_after_seq(k_matrix, seq_of_k_to_id: dict, seq: str, alpha: float = 0) -> str:
    """
    Given sequence of last k tokens, samples next token
    :param seq:
    :param alpha: Creativity hyperparameter, large the value => distribution approches uniormity => choice of random token
    :return:
    """
    if seq not in seq_of_k_to_id:
        # markov chains has not seen any following token to this sequence
        return random.choice(tokens)
    next_token_vector = k_matrix[seq_of_k_to_id[seq]] + alpha
    likelihoods = next_token_vector/next_token_vector.sum()
    return random.choices(tokens, likelihoods.toarray()[0])[0]

def stochastic_chain(k_matrix, seq_of_k_to_id: dict, seed: str = None, chain_length: int = 15):
    """
    Returns sentence with added tokens equal to 'chain_length'.
    The last k-token phrase of the seed sentence need to be present in some part of the training dataset.

    :param seed: If None, randomly generates seed sentence.
    :param chain_length: Added tokens to be generated.
    :return: seed sentence + generated sentence.
    """
    k = len(tokenize_text(list(seq_of_k_to_id.keys())[0]))
    if seed is None:
        seed = random.choice(list(seq_of_k_to_id.keys()))
    current_tokens = tokenize_text(seed)
    generated_tokens = current_tokens
    if len(current_tokens) < k:
        raise ValueError(f"Seed length must be at-least as long as {k}")
    current_tokens = current_tokens[-k:]
    if seq_of_k_to_id.get(textify_tokens(current_tokens), None) is None:
        raise ValueError("The last k-word phrase of the seed sentence need to be present in some part of the training dataset.")
    for _ in range(chain_length):
        next_token = sample_next_token_after_seq(k_matrix, seq_of_k_to_id, seq=textify_tokens(current_tokens))
        generated_tokens.append(next_token)
        current_tokens = current_tokens[1:] + [next_token]
    sentence = textify_tokens(generated_tokens)
    sentence = postprocess_text(sentence)
    return sentence

In [9]:
k_list = list(range(1, 7))
# k_list = [1]
k_calculated = {}
for k in k_list:
    k_matrix, seq_of_k_to_id = get_k_matrix(k=k)
    print(f"{len(seq_of_k_to_id)} \tmany {k}-token-sequences have been mapped to {vocab_size} tokens.")
    k_calculated[k] = (k_matrix, seq_of_k_to_id)

14343 	many 1-token-sequences have been mapped to 14343 tokens.
106460 	many 2-token-sequences have been mapped to 14343 tokens.
210180 	many 3-token-sequences have been mapped to 14343 tokens.
270300 	many 4-token-sequences have been mapped to 14343 tokens.
299377 	many 5-token-sequences have been mapped to 14343 tokens.
312964 	many 6-token-sequences have been mapped to 14343 tokens.


In [11]:
for k, calculated in k_calculated.items():
    k_matrix, seq_of_k_to_id = calculated
    generated = stochastic_chain(
        k_matrix=k_matrix,
        seq_of_k_to_id=seq_of_k_to_id,
        seed=None,
        chain_length=80
    )
    print(f"\n------------------------- GENERATED SENTENCE -- k = {k}")
    print(generated)
    print("------------------------- END")


------------------------- GENERATED SENTENCE -- k = 1
preparation. 
 Reignier, Clarence did I will, 
 And welcome home again, in thy earliness doth give thee for my soul, 

 BAPTISTA: Come, make fit his troops, 
 If you of woe obey. 
 Thanks, wagery, lords, on Edward hath look' dst Polixenes; then I have goaded onward. 
 So season. 

 Is very Mab


 High- a state
------------------------- END

------------------------- GENERATED SENTENCE -- k = 2
s rattling bones, 
 The gods grant them true! 

 RICHARD: 
 These English woes will make them sharp, and, sooth to say I said loose- bodied and the time! 
 Henceforward do your best haste, his blood committed to your hand too much. Servants, leave me but love' s fight with none but thou talk' d Jove' s tooth doth never rankle more
 Than in my
------------------------- END

------------------------- GENERATED SENTENCE -- k = 3
near, be ne' er so fair, and I' ll pray a thousand prayers for thy death: 
' Tis he, that should be husband, comes to 